### Initialization

In [2]:
# built-in
import time
from datetime import datetime
import json
import re

# third-party
import pandas as pd
import bitalino
import serial
import scientisst
import numpy as np
import plotly.express as px
import biosppy as bp
from plotly.subplots import make_subplots
import plotly.graph_objects as go

color_palette=['#C98986', '#8B575C', '#7899D4', '#9EC1A3', '#1F363D'] 

In [20]:
# Initiate variables

timeout = 120
batch_size = 100
sampling_rate = 100

arduino_pattern = re.compile(r'\d+\n') # pattern to make sure the message received from the serial port is in the format "{value}\n"

#bitalino_args = {"mac_address": "/dev/tty.BITalino-6E-27", "sampling_rate": sampling_rate, "channels": [1], "batch_size": batch_size}
scientisst_args = {"mac_address": "/dev/cu.ScientISST-32-22", "sampling_rate": sampling_rate, "channels": [1,2,3,4,5,6], "batch_size": batch_size}
#arduino_args = {"port": "/dev/cu.usbserial-0001", "sampling_rate": sampling_rate, "batch_size": batch_size}

### Define auxiliary functions

In [4]:
# Define SerialPortUnavailable exception
from serial.tools import list_ports

class SerialPortUnavailable():

    def __init__(self):
        print("Chosen port is not available. Choose one of the following ports:")
        for port in list_ports.comports():
            print(f"    * {port.name}")


SerialPortUnavailable()

Chosen port is not available. Choose one of the following ports:
    * cu.wlan-debug
    * cu.Bluetooth-Incoming-Port
    * cu.ScientISST-54-96
    * cu.ScientISST-32-22


In [5]:
# Define class that hold all setup informations about the acquisition

class Acquisition:

    def __init__(self, start_time, file):
        
        self.sampling_rate = sampling_rate
        self.keep_alive = True
        self.devices_started = False
        self.bitalino_args = bitalino_args
        self.scientisst_args = scientisst_args
        #self.arduino_args = arduino_args
        self.start_time = start_time
        self.file = file

In [6]:
# Define function that creates file header 

def get_header(acquisition):

    scientisst_channels = ""
    for channel in acquisition.scientisst_args["channels"]:
        scientisst_channels += f"\tscientisst_raw_AI{channel}"
    
    bitalino_channels = ""
    for channel in acquisition.bitalino_args["channels"]:
        bitalino_channels += f"\tbitalino_raw_A{channel}"

    return f"#scientisst_nseq{scientisst_channels}\n"

In [7]:
# Define Exceptions

class TimeoutException(Exception):
    "   Connection attempt timed out"
    pass

class ConnectionFailedException(Exception):
    "   Failed to connect to device"
    pass

class UnknownDeviceException(Exception):
    "   Unknown device - please choose ScientISST, BITalino or Arduino"
    pass

### Define functions for device connection

In [8]:
def connect_device(address, device_type):

    device = None
    init_connect_time = time.time()
    print(f'Searching for {device_type}... {address}')

    while (time.time() - init_connect_time) < timeout:

        try:
            if device_type == "BITalino":
                device = bitalino.BITalino(address)
                if device.macAddress:
                    print("Connected!")
                    return device
            elif device_type == "ScientISST":
                device = scientisst.ScientISST(address)
                if device.address:
                    return device
            elif device_type == "Arduino":
                device = serial.Serial(address, baudrate=115200, timeout=0)
                print("Connected!")
                return device
            else:
                time.sleep(0.1)
                raise UnknownDeviceException

        except Exception as e:
            print(e)
            continue

    raise TimeoutException

### Define functions to start and read devices

In [9]:
def start_devices(bitalino_device, scientisst_device, acquisition):

    scientisst_device.start(sample_rate=acquisition.scientisst_args["sampling_rate"], channels=acquisition.scientisst_args["channels"])  
    #bitalino_device.start(SamplingRate=acquisition.bitalino_args["sampling_rate"], analogChannels=acquisition.bitalino_args["channels"])
    #arduino_device = connect_device(acquisition.arduino_args["port"], "Arduino")
    
    #return arduino_device

In [45]:
def read_batch(bitalino_device, scientisst_device, arduino_device, acquisition):
    
    try: 
        
        scientisst_data = scientisst_device.read(matrix=True)
        scientisst_data = np.take(scientisst_data,[0]+[5+(chn-1)*2 for chn in acquisition.scientisst_args["channels"]], axis=1)
        #bitalino_data = bitalino_device.read(scientisst_data.shape[0])
        #bitalino_data = np.take(bitalino_data,[0]+[*range(-1, -len(acquisition.bitalino_args["channels"])-1, -1)], axis=1)

        # arduino_data = []
        # arduino_message = ""
        # match = None

        # while len(arduino_data) < scientisst_data.shape[0]:
        #     arduino_bytes = arduino_device.readline().decode() # read message from arduino 

        #     if arduino_bytes != arduino_message:
        #         arduino_message = arduino_message + arduino_bytes # in case the full line comes in different messages, concatenate it
        #         match = arduino_pattern.search(arduino_message) # make sure the message received from the serial port is in the format "{value}\n"

        #     if match is not None: 
        #         arduino_data += [[int(match.group().strip())]]
        #         arduino_message = ""
                
        #print(f"bitalino data:\n{bitalino_data}")
        #print(f"\n\nscientisst data:\n{scientisst_data}")
        #print(f"\n\narduino data:\n{np.array(arduino_data)}")
    
        #data = np.hstack((scientisst_data, np.array(arduino_data)))
        print(f"\ndata: {scientisst_data}")

        np.savetxt(acquisition.file, scientisst_data, fmt='\t'.join(['%d']*scientisst_data.shape[1]))
        
    except Exception as e:
        print(e)

### Main script

In [48]:
## Initialization

try: 
    
    now = datetime.now()
    file_path = f"./data/sample_{now.strftime('%d-%m-%Y_%H-%M-%S')}.txt"
    file = open(file_path,"w")
    acquisition = Acquisition(time.time(), file)

    file.write(get_header(acquisition))

    # Connect devices
    scientisst_device = connect_device(scientisst_args["mac_address"], "ScientISST")
    bitalino_device = None #bitalino_device = connect_device(bitalino_args["mac_address"], "BITalino")
    arduino_device = None # Connection to this device happens at a later stage

except KeyboardInterrupt:
    print(f"Script terminated before acquisition was started")
    acquisition.keep_alive = False
except ConnectionFailedException:
    acquisition.keep_alive = False
except UnknownDeviceException:
    acquisition.keep_alive = False
except Exception as e:
    print(e)
    acquisition.keep_alive = False


## Device streaming

try: 

    while acquisition.keep_alive:
        
        if not acquisition.devices_started: 
            # If the devices have not yet started acquiring or they are paused, start acquisition
            try: # TODO: timeout para isto
                start_devices(bitalino_device, scientisst_device, acquisition)
                acquisition.devices_started = True

            except Exception as e:
                print(e)
                pass          
        
        else:
            try:
                read_batch(bitalino_device, scientisst_device, arduino_device, acquisition)
            except KeyboardInterrupt:
                acquisition.keep_alive = False
                print(f"\n\nAcquisition terminated")

            except:
                #bitalino_device.stop()
                scientisst_device.stop()
                #arduino_device.close()
                acquisition.devices_started = False
                pass


except KeyboardInterrupt:
    print(f"\n\nAcquisition terminated")
    
except Exception as e:
    print(e)
    #bitalino_device.stop()
    scientisst_device.stop()
    #arduino_device.close()
    file.close()

#bitalino_device.stop()
scientisst_device.stop()
#arduino_device.close()
file.close()
print("here")


Searching for ScientISST... /dev/cu.ScientISST-32-22
Connecting to /dev/cu.ScientISST-32-22...
ScientISST version: 1.0
ScientISST Board Vref: 1107
ScientISST Board ADC Attenuation Mode: 0
Connected!

data: [[  0   0 573 765 752   0   0]
 [  1   0 573 761 755   0   0]
 [  2   0 572 762 752   0   0]
 [  3   0 574 764 757   0   0]
 [  4   0 571 762 755   0   0]
 [  5   0 571 762 753   0   0]
 [  6   0 570 762 754   0   0]
 [  7   0 571 764 752   0   0]
 [  8   0 573 763 752   0   0]
 [  9   0 572 766 753   0   0]
 [ 10   0 571 763 753   0   0]
 [ 11   0 577 761 752   0   0]
 [ 12   0 570 763 753   0   0]
 [ 13   0 569 763 752   0   0]
 [ 14   0 570 763 751   0   0]
 [ 15   0 571 765 752   0   0]
 [  0   0 571 762 757   0   0]
 [  1   0 573 766 752   0   0]
 [  2   0 567 758 752   0   0]
 [  3   0 573 759 752   0   0]]

data: [[  4   0 570 761 752   0   0]
 [  5   0 571 762 753   0   0]
 [  6   0 573 766 753   0   0]
 [  7   0 573 762 753   0   0]
 [  8   0 572 762 753   0   0]
 [  9   0 5

### Read and plot data

In [56]:
# Read ScientISST data

#scientisst_file_path = "data/partial-abduction_scientisst_11-01-2023_11-29-41.txt"

scientisst_file_path = file_path

scientisst_data = pd.read_csv(scientisst_file_path, header=0, delimiter="\t", skipfooter=1, engine='python')
scientisst_data[f"scientisst_filtered_AI6"], _, _ = bp.signals.tools.filter_signal(signal=scientisst_data[f"scientisst_raw_AI6"].to_numpy(), ftype='FIR', band='lowpass', order=8, frequency=1, sampling_rate=sampling_rate)

px.line(scientisst_data, y=["scientisst_raw_AI2"], color_discrete_sequence=color_palette)

In [15]:
# Read BITalino data

bitalino_file_path = "data/partial-abduction_bitalino_11-01-2023_11-29-54.txt"

bitalino_data = pd.read_csv(bitalino_file_path, header=0, delimiter="\t", skipfooter=1, engine='python')
bitalino_data[f"bitalino_filtered_A1"], _, _ = bp.signals.tools.filter_signal(signal=bitalino_data[f"bitalino_raw_A1"].to_numpy(), ftype='FIR', band='lowpass', order=8, frequency=1, sampling_rate=10.0)

px.line(bitalino_data, y=["bitalino_filtered_A1"])


### Get annotations

In [ ]:
annots = scientisst_data.index[scientisst_data['arduino_raw'] == False].tolist()

annots_new = []
last_annot = -2
for annot in annots:
    if annot != last_annot + 1:
        annots_new += [annot]
    last_annot = annot

### Synchronize and plot signals

In [16]:
def find_k_peaks(sig, mode='min', k=2):
    # Returns indexes of k "peaks", sorted according to magnitude of peaks

    peaks_idx, peaks_vals = bp.signals.tools.find_extrema(sig, mode=mode)
    if mode == "both":
        k_values = np.argsort(abs(peaks_vals.values))[-k:]
    elif mode == "min":
        k_values = np.argsort(peaks_vals)[:k]
    elif mode == "max":
        k_values = np.argsort(peaks_vals)[-k:]
    k_peaks = peaks_idx[k_values]
    return k_peaks



def synchronize(signal1, signal2, signal1_name, signal2_name, detrend=True, plot=True):
    # Returns dataframe with synchronized signals and relative timestamp as index

    # Detrend input signals by removing the mean
    detrend_signal1 = (signal1 - np.mean(signal1)) / np.std(signal1)
    detrend_signal2 = (signal2 - np.mean(signal2)) / np.std(signal2)
    
    # Find max inspiration peaks and invert signals if necessary
    # peaks1 = find_k_peaks(detrend_signal1[:100], mode="both")
    # if detrend_signal1[peaks1[0]] > 0:
    #     signal1 = -signal1
    #     detrend_signal1 = -detrend_signal1
    # peaks2 = find_k_peaks(detrend_signal2, mode="both")
    # if detrend_signal2[peaks2[0]] > 0:
    #     signal2 = -signal2
    #     detrend_signal2 = -detrend_signal2

    peaks1 = find_k_peaks(detrend_signal1[:100], mode="max")
    peaks2 = find_k_peaks(detrend_signal2[:100], mode="max")

    if detrend:
        signal1 = detrend_signal1
        signal2 = detrend_signal2

    # Plot peaks found
    if plot:
        fig = make_subplots(rows=2, cols=1)
        fig.add_trace(go.Scatter(y=signal1, marker_color=color_palette[0], name=signal1_name),
            row=1, col=1)
        fig.add_trace(go.Scatter(y=signal2, marker_color=color_palette[1], name=signal2_name),
            row=2, col=1)
        for peak in peaks1:
            fig.add_vline(x=peak, line_color=color_palette[0])
        for peak in peaks2:
            fig.add_vline(x=peak, line_color=color_palette[1])
        fig.show()

    # Find delay (in indexes) and trim signals accrodingly
    n_peaks = min(len(peaks1), len(peaks2))
    delay = int(np.mean(np.sort(peaks1)[:n_peaks] - np.sort(peaks2)[:n_peaks]))

    if delay > 0:
        signal1 = signal1[delay:]
    else:
        signal2 = signal2[-delay:]

    trim_len = min(len(signal1), len(signal2))
    signal1 = signal1[:trim_len]
    signal2 = signal2[:trim_len]

    sync_data = pd.DataFrame({signal1_name: np.array(signal1), signal2_name: np.array(signal2)})
    if plot:
        fig = px.line(sync_data, color_discrete_sequence=color_palette)
        fig.show()

    return sync_data


sync_data = synchronize(scientisst_data[f"scientisst_filtered_AI6"], bitalino_data[f"bitalino_filtered_A1"], "scientisst", "bitalino")